In [ ]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os


In [ ]:
mp_pose = mp.solutions.pose


In [ ]:
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

col_names = []
for i in range(len(landmarks_points.keys())):
    name = list(landmarks_points.keys())[i]
    col_names.append(name + "_x")
    col_names.append(name + "_y")
    col_names.append(name + "_z")
    col_names.append(name + "_v")
col_names.append("pose")


In [ ]:
full_list = []

main_dir = "./DATASET/TRAIN"
pose_dir_list = os.listdir(main_dir)

for j in range(0, len(pose_dir_list)):
    dir_list = os.listdir(f"{main_dir}/{pose_dir_list[j]}")

    for i in range(0, len(dir_list)):

        pre_list = []

        with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
            image = cv2.imread(f"{main_dir}/{pose_dir_list[j]}/{dir_list[i]}")
            result = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            try:
                for landmarks in result.pose_landmarks.landmark:
                    pre_list.append(landmarks)
            except AttributeError:
                print(
                    f"No points on {main_dir}/{pose_dir_list[j]}/{dir_list[i]}")

        gen1116 = np.array([[pre_list[k].x, pre_list[k].y, pre_list[k].z,
                           pre_list[k].visibility] for k in range(11, 17)]).flatten().tolist()

        gen2333 = np.array([[pre_list[k].x, pre_list[k].y, pre_list[k].z,
                           pre_list[k].visibility] for k in range(23, 33)]).flatten().tolist()

        gen1116.extend(gen2333)

        all_list = [
            pre_list[0].x,
            pre_list[0].y,
            pre_list[0].z,
            pre_list[0].visibility,
        ]

        all_list.extend(gen1116)
        all_list.append(j)
        full_list.append(all_list)


In [289]:
data = pd.DataFrame(full_list, columns=col_names)
data.to_csv("./DATASET/data.csv", index=False)
